# Imports

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import matplotlib.pyplot
from mpl_finance import candlestick_ohlc
import dateutil.parser as dparser
import numpy as np

# Helpers

In [ ]:
def zipp(list1, list2):
    a = len(list1)
    b = len(list2)
    return [[list1[i], list2[i]] for i in range(min(a, b))]
        

# Main Functions

In [ ]:
def search(query, n, currentyear):
    """This functions scrapes the NASDAQ newsfeed for article links """
    Pairs = []
    for i in range(1, n + 1):
        if i == 1:
            url = "https://www.nasdaq.com/symbol/" + query + "/news-headlines"
        else:
            url = "https://www.nasdaq.com/symbol/" + query + "/news-headlines?page=" + str(i)
    
        page = requests.get(url)

        soup = BeautifulSoup(page.content, 'html.parser')
    
        links = soup.findAll('a')
        dates = soup.findAll('small')
    
        Article_list = []
        Date_list = []
    
        """
        Get all the links for the articles from the NASDAQ newsfeed
        """
        counter = 0
    
        for link in links:
            article = link.get('href')
            if ("/article/" in article):
                if counter > 5:
                    Article_list.append(article)
                counter += 1

        """
        Get all the dates for the articles
        """
    
        for date in dates:
            text = date.get_text()
            if "/" + currentyear in text:
                Date_list.append(text[24:33])
    
        Pairs.extend(zipp(Article_list, Date_list))
    """
    # Print suite for debugging
    for p in Pairs:
        print(p)
    print(len(Pairs))
    """
    
    return Pairs


In [ ]:
def count_keywords(keywords, url):
    page = requests.get(url)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    
    text = soup.get_text().lower()
    
    number = 0
    for k in keywords:
        number += len(re.findall(r'(?=' + k.lower() + ')', text))
    return number


In [ ]:
def process_data(Pairs, keywords):
    plot_vals = []
    for p in Pairs:
        new_pair = [dparser.parse(p[1], fuzzy=True), count_keywords(keywords, p[0])]
        plot_vals.append(new_pair)
    return plot_vals


In [ ]:
def get_stock_data(query):
    url = "https://www.nasdaq.com/symbol/" + query + "/historical"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    stock_data = []
    
    chart = soup.find("tbody")
    days = chart.findAll("tr")
    for day in days:
        data = day.findAll("td")
        text_data = [d.get_text() for d in data]
        clean_data = [text[35:].split('\r')[0] for text in text_data]
        
        stock_data.append(clean_data)
        
    stock_data = stock_data[1:]
    for s in stock_data:
            s[0] = dparser.parse(s[0])
            for i in range(1,5):
                s[i] = float(s[i])
            s[5] = s[5].replace(',','')
            s[5] = int(s[5])
    return stock_data
    

In [ ]:
def sum_counts(plot_vals):
    new_date_list = []
    for p in plot_vals:
        if p[0] not in new_date_list:
            new_date_list.append(p[0])
    size = len(new_date_list)
    new_count_list = [0]*size
    for i in range(size):
        for p in plot_vals:
            if new_date_list[i] == p[0]:
                new_count_list[i] += p[1]
    return zipp(new_date_list, new_count_list)


In [ ]:
def keyword_scrape(query, keywords, n=1, currentyear="2019"):
    data = search(query, n, currentyear)
    processed_data = process_data(data, keywords)
    return sum_counts(processed_data)

In [ ]:
data = keyword_scrape("AAPL", ["new", "tablet"], n=3)
stock = get_stock_data("AAPL")

# Graphing

In [ ]:
# plot keyword occurences vs date
def plot_keyword_occurences(data):
    xlist = [d[0] for d in data]
    ylist = [d[1] for d in data]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    plt.bar(xlist,ylist)
    plt.gcf().autofmt_xdate()

In [ ]:
# plot stock data for past 3 months
# stock data = [date, open, high, low, close, volume]

import matplotlib.dates as mdates

def plot_candles(stock):
    # option is int 0 to 4
    stock_data = [[mdates.date2num(s[0]), s[1], s[2], s[3], s[4]] for s in stock]
    stride = 4
    fig = plt.figure(figsize=[25,10], dpi=300)
    
    ax1 = fig.add_subplot(111)

    candlestick_ohlc(ax1,stock_data,width=0.8,colorup='#77d879', colordown='#db3f3f') 
    
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    plt.gcf().autofmt_xdate()
    
    for n, label in enumerate(ax1.xaxis.get_ticklabels()):
        if n % stride != 0:
            label.set_visible(False)
            
def plot_volume(stock):
    dates = [s[0] for s in stock]
    volume_data = [s[5] for s in stock]
    
    fig = plt.figure(figsize=[25,10], dpi=300)
    
    ax1 = fig.add_subplot(111)
    
    plt.bar(dates, volume_data)
    
    stride = 4
    
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    plt.gcf().autofmt_xdate()
    
    for n, label in enumerate(ax1.xaxis.get_ticklabels()):
        if n % stride != 0:
            label.set_visible(False)

# Testing

In [ ]:
plot_keyword_occurences(data)

In [ ]:
plot_stocks(stock)

In [ ]:
plot_volume(stock)